In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [22]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_simple_lstm_ru'
!mkdir {pathjoin(MODELS_DIR, MODEL_ID)}
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [3]:
transformer_model = 'DeepPavlov/rubert-base-cased'
train_dataset_reader = build_dataset_reader(None, lower=True)
val_dataset_reader = build_dataset_reader(None, lower=True)

train_data, dev_data = read_data(
    pathjoin(DATA_DIR, "ru_train"), 
    pathjoin(DATA_DIR, "ru_test"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data
<class 'data_processing.ClassificationDatasetReader'> /home/mlepekhin/data/ru_train



Building the vocabulary


In [23]:
model = build_simple_lstm_model(vocab, num_layers=1, emb_size=256, hidden_size=128)

Building the model


In [24]:
if torch.cuda.is_available():
    cuda_device = 3
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

3


In [25]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [26]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device,
    learning_rate=0.001,
)
print("Starting training")
trainer.train()
print("Finished training")

Starting training



Finished training


In [28]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
trainer.model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

rm: cannot remove '/home/mlepekhin/models/allennlp_simple_lstm_ru/checkpoints/*.json': No such file or directory
rm: cannot remove '/home/mlepekhin/models/allennlp_simple_lstm_ru/checkpoints/model*': No such file or directory
rm: cannot remove '/home/mlepekhin/models/allennlp_simple_lstm_ru/checkpoints/training*': No such file or directory


total 186M
-rw-rw-r-- 1 mlepekhin mlepekhin 186M сен 23 17:22 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K сен 23 17:20 log


In [9]:
!gpustat

seven                Tue Sep 15 19:52:21 2020  440.33.01
[0] TITAN RTX        | 87'C,  94 % | 15395 / 24220 MB | sharuev(15383M)
[1] TITAN RTX        | 90'C,  94 % | 17513 / 24220 MB | sharuev(17501M)
[2] TITAN RTX        | 77'C,  95 % | 11375 / 24220 MB | sharuev(11363M)
[3] TITAN RTX        | 84'C,   4 % | 19435 / 24220 MB | mlepekhin(19423M)
